In [1]:
""" Starter code for simple logistic regression model for MNIST
with tf.data module
MNIST dataset: yann.lecun.com/exdb/mnist/
Created by Chip Huyen (chiphuyen@cs.stanford.edu)
CS20: "TensorFlow for Deep Learning Research"
cs20.stanford.edu
Lecture 03
"""
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
import time

import utils

In [17]:
# Define paramaters for the model
learning_rate = 0.02
batch_size = 128
n_epochs = 50
n_train = 60000
n_test = 10000

In [5]:
# Step 1: Read in data
mnist_folder = 'data/not-mnist'
#utils.download_mnist(mnist_folder)
train, val, test = utils.read_mnist(mnist_folder, flatten=True)

In [6]:
# Step 2: Create datasets and iterator
# create training Dataset and batch it
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(500) # if you want to shuffle your data
train_data = train_data.batch(batch_size)

In [7]:
# create testing Dataset and batch it
test_data = tf.data.Dataset.from_tensor_slices(test)
test_data = test_data.batch(batch_size)

In [8]:
# create one iterator and initialize it with different datasets
iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
img, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)	# initializer for train_data
test_init = iterator.make_initializer(test_data)	# initializer for train_data

In [9]:
train_data.output_shapes[0]

TensorShape([Dimension(None), Dimension(784)])

In [10]:
# Step 3: create weights and bias
# w is initialized to random variables with mean of 0, stddev of 0.01
# b is initialized to 0
# shape of w depends on the dimension of X and Y so that Y = tf.matmul(X, w)
# shape of b depends on Y
w = tf.get_variable(shape=(784, 10), name="w", initializer=tf.random_uniform_initializer(0, 0.01))
b = tf.get_variable(shape=(1, 10), name="b", initializer=tf.zeros_initializer())

In [11]:
# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer
logits = tf.matmul(img, w) + b

In [12]:
# Step 5: define loss function
# use cross entropy of softmax of logits as the loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=label), name="loss")

In [13]:
# Step 6: define optimizer
# using Adamn Optimizer with pre-defined learning rate to minimize loss
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [18]:
# Step 7: calculate accuracy with test set
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

writer = tf.summary.FileWriter('./graphs/logreg', tf.get_default_graph())
with tf.Session() as sess:
   
    start_time = time.time()
    sess.run(tf.global_variables_initializer())

    # train the model n_epochs times
    for i in range(n_epochs): 	
        sess.run(train_init)	# drawing samples from train_data
        total_loss = 0
        n_batches = 0
        try:
            while True:
                _, l = sess.run([optimizer, loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
    print('Total time: {0} seconds'.format(time.time() - start_time))

    # test the model
    sess.run(test_init)			# drawing samples from test_data
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass

    print('Accuracy {0}'.format(total_correct_preds/n_test))
writer.close()

Average loss epoch 0: 0.7822071261877237
Average loss epoch 1: 0.7350290422522745
Average loss epoch 2: 0.7346450470214666
Average loss epoch 3: 0.7331762111464212
Average loss epoch 4: 0.7277874517579411
Average loss epoch 5: 0.7360644364079764
Average loss epoch 6: 0.7217312092004821
Average loss epoch 7: 0.731551160507424
Average loss epoch 8: 0.7142372374617776
Average loss epoch 9: 0.7386562434046767
Average loss epoch 10: 0.7109834635673568
Average loss epoch 11: 0.7358285506104314
Average loss epoch 12: 0.7175513536431068
Average loss epoch 13: 0.7146795205598654
Average loss epoch 14: 0.7083705633185631
Average loss epoch 15: 0.7155892591836841
Average loss epoch 16: 0.7123416190923646
Average loss epoch 17: 0.7120157774797705
Average loss epoch 18: 0.7084919637025788
Average loss epoch 19: 0.6938195307587468
Average loss epoch 20: 0.7048720271781433
Average loss epoch 21: 0.7071929438169613
Average loss epoch 22: 0.711730506738951
Average loss epoch 23: 0.7120716036752213
Aver